In [1]:
# pip install packages
%pip uninstall openai --yes
%pip install openai==0.28
%pip install tiktoken
%pip install pdfminer
%pip install pdfminer.six
%pip install docx2txt
%pip install python-pptx
# %pip install pinecone-client==2.2.4
%pip install pinecone
%pip install pinecone-client --upgrade



# imports
import openai
#from openai import OpenAI
import os
import tiktoken
from tqdm.auto import tqdm     # this is our progress bar
import pinecone
import re
import pandas as pd
from pdfminer.high_level import extract_text, extract_pages
import docx2txt
from pptx import Presentation
import zipfile
import xml.etree.ElementTree as ET
import base64



# constants
GPT_MODEL = 'gpt-4o'
EMBEDDING_MODEL = 'text-embedding-ada-002'

# this is for the input data
PATH = 'data\\temp-financial'

# these are Pinecone vars
INDEX = 'idx'

ENV = os.getenv("ENV")



# env vars
%env environment=ENV


# store keys for OpenAI and Pinecone
#openai.api_key = os.environ['OPENAI_API_KEY']
#print(f"Here is your OpenAI API Key: {openai.api_key}")
# print(f"Here is your Pinecone API Key: {os.environ['PINECONE_API_KEY']}")

Found existing installation: openai 0.28.0
Uninstalling openai-0.28.0:
  Successfully uninstalled openai-0.28.0
Note: you may need to restart the kernel to use updated packages.
  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
env: environment=ENV


/opt/homebrew/Cellar/jupyterlab/4.2.1/libexec/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%pip install python-dotenv

from dotenv import load_dotenv

load_dotenv()

#%env OPENAI_API_KEY=os.getenv('OPENAI_API_KEY')
%env PINECONE_API_KEY=os.getenv('PINECONE_API_KEY')
openai.api_key = os.getenv('OPENAI_API_KEY')


Note: you may need to restart the kernel to use updated packages.
env: PINECONE_API_KEY=os.getenv('PINECONE_API_KEY')


In [3]:
def num_tokens(text: str, model: str = 'gpt-4') -> int:
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

In [4]:
def read_txt(path):
    with open(path, encoding='utf8') as f:
        return f.readlines()

In [5]:
def read_pdf(path):
    return extract_text(path).split('\n')

In [6]:
def read_docx(path):
    # extract text
    return docx2txt.process(path).split('\n')

In [7]:
def read_pptx(path):
    split = []
    prs = Presentation(path)
    print("----------------------")
    temp = ''
    enterCount = 0
    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, 'text'):
#                 if len(shape.text) > 0:
#                     temp += shape.text
#                 elif enterCount > 1:
#                     split.append(temp)
#                     temp = ''
#                     enterCount = 0
#                 elif len(shape.text) == 0:
#                     enterCount += 1
                split.append(shape.text)
    
    return split

In [8]:
def read_xlsx(path):
    with zipfile.ZipFile(path, 'r') as zip_ref:
        zip_ref.extractall(PATH)
    
    xml_file = os.path.join(PATH, 'xl/sharedStrings.xml')
    
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    result = []
    
    for elem in root.iter():
        if elem.text:
            result.append(elem.text.strip())
    
    return result

In [9]:
def read_png(path):
    with open(path, "rb") as image_file:
        image = base64.b64encode(image_file.read()).decode("utf-8")
        
        openai.api_key = os.environ['OPENAI_API_KEY']
        
        messages = [
            { 'role': 'system', 'content': 'system message here' },
            { 'role': 'user', 'content': [
                {'type': 'text', 'text': f'Convert the following image into a text table.'},
                {'type': 'image_url', 'image_url': { 'url': f'data:image/png;base64,{image}'} }
            ]}
        ]
        
        functions = [
            {
                'name': 'write_queries',
                'description': 'Get information from the database',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'details': {
                            'type': 'string',
                            'description': 'The information needed from the database',
                        },
                    }
                }
            }
        ]
        
        result = ''
        
        for chunk in openai.chat.completions.create(
            model=GPT_MODEL,
            messages=messages,
            functions=functions,
            temperature=0,
            frequency_penalty=0,
            presence_penalty=0,
            function_call=None,
            stream=True
        ):
            if len(chunk.choices) > 0:
                response = chunk.choices[0]
                finish_reason = response.finish_reason
                content = response.delta.content
                function = response.delta.function_call

                if content is not None:
                    print(content, end='')
                    result += content
        
        return result.split('\n')

In [10]:
def collect_all_files(root_path):
    all_files = []
    for root, dirs, files in os.walk(root_path):
        for file in files:
            file_path = os.path.join(root, file)
            all_files.append(file_path)
    return all_files

In [11]:
def get_range(s, delimiter = '.'):
    parts = s.split(delimiter)
    return delimiter.join(parts[-1:])

In [12]:
messages = []

files = collect_all_files(PATH)

lines = []
for file in files:
    lines = []
    print(file)
    suffix = get_range(file)
    if suffix == 'txt':
        lines = read_txt(file)
    elif suffix == 'pdf':
        lines = read_pdf(file)
    elif suffix == 'docx':
        lines = read_docx(file)
    elif suffix == 'pptx':
        lines = read_pptx(file)
    elif suffix == 'xlsx':
        lines = read_xlsx(file)
    elif suffix == 'png':
        lines = read_png(file)

    temp_string = []
    print(lines)
    for line in lines:
        if num_tokens('\n'.join(temp_string)) > 256:
            print('\n'.join(temp_string))
            messages.append('\n'.join(temp_string))
            temp_string = []
        else:
            temp_string.append(line)
    if num_tokens('\n'.join(temp_string)) < 256:
        print('\n'.join(temp_string))
        messages.append('\n'.join(temp_string))

In [13]:
messages = [message for message in messages if len(message) > 0]

In [14]:
if not messages: 
    messages = ["this is message 1", "this is message 2", "this is message 3", "this is message 4", "meow meow", "woof woof", "I am a cat"]

print(messages)

['this is message 1', 'this is message 2', 'this is message 3', 'this is message 4', 'meow meow', 'woof woof', 'I am a cat']


In [15]:
from pinecone import Pinecone, PodSpec

INDEX = 'idx'

pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])

if INDEX not in pc.list_indexes().names():
    pc.create_index(INDEX, dimension=1536, metric='cosine', spec=PodSpec(environment=ENV))
# connect to index
index = pc.Index(INDEX)

In [16]:
# calculate embeddings
BATCH_SIZE = 128  # you can submit up to 2048 embedding inputs per request

embeddings = []
for batch_start in range(0, len(messages), BATCH_SIZE):
    batch_end = batch_start + BATCH_SIZE
    batch = messages[batch_start:batch_end]
    print(f"Batch {batch_start} to {batch_end-1}")
    response = openai.Embedding.create(model=EMBEDDING_MODEL, input=batch)
    for i, be in enumerate(response['data']):
        assert i == be['index']  # double check embeddings are in same order as input
    batch_embeddings = [e['embedding'] for e in response['data']]
    embeddings.extend(batch_embeddings)

df = pd.DataFrame({'text': messages, 'embedding': embeddings})

Batch 0 to 127


In [17]:
batch_size = 32  # process everything in batches of 32
for i in tqdm(range(0, len(df['text']), batch_size)):
    # set end position of batch
    i_end = min(i+batch_size, len(df['text']))
    # get batch of lines and IDs
    lines_batch = df['text'][i:i+batch_size]
    embeds = df['embedding'][i:i+batch_size]
    ids_batch = ['training:' + str(n) for n in range(i, i_end)]
    # prep metadata and upsert batch
    meta = [{'text': line} for line in lines_batch]
    to_upsert = zip(ids_batch, embeds, meta)
    # upsert to Pinecone
    index.upsert(vectors=list(to_upsert))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.23it/s]


In [18]:
embeddings = df.to_dict()

In [19]:
def send_embeddings(query: str = '', embeddings: dict = {}, max_tokens: int = 1024) -> str:
    '''Return the max_tokens amount of related contexts based on the query string.

    Keyword arguments:
    query -- The query needing context
    embeddings -- The list of embeddings and text
    max_tokens -- The number of tokens to pull (Default 1024)

    Returns:
    The context of the query
    '''
    embedding = openai.Embedding.create(model=EMBEDDING_MODEL, input=query)['data'][0]['embedding']
    context = [message for _, message in sorted(zip(cosine_similarity(embeddings['embedding'], [embedding]), embeddings['text']), reverse=True)]

    max_tokens = int(max_tokens)

    text = ''
    total_tokens = 0
    max_tokens = min(num_tokens('\n'.join(context)) + total_tokens + 1, max_tokens + 1)
    i = 0

    for item in context:
        nxt = f'\n{item}\n'
        total_tokens += num_tokens(nxt)
        if total_tokens > max_tokens:
            break
        text += nxt
        i += 1

    return f'{text}'

In [20]:
chatbox = ''
while chatbox != 'exit':
    chatbox = input()
    
    if chatbox == 'exit':
        continue
    
    index = "idx"

    # client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

    # STEP 1: Embed your prompt
    embedding = openai.Embedding.create(model=EMBEDDING_MODEL, input=chatbox).data[0].embedding

    # STEP 2: Initialize pinecone index
    pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])

    if index in pc.list_indexes().names():
        idx = pc.Index(index)
        result = idx.query(vector=[embedding], top_k=100, include_metadata=True)
        context = [x['metadata']['text'] for x in result['matches']]

        print(context)

 bipedal animal


['woof woof', 'I am a cat', 'meow meow', 'this is message 2', 'this is message 1', 'this is message 4', 'this is message 3']


 quadriped


['woof woof', 'this is message 4', 'meow meow', 'this is message 3', 'I am a cat', 'this is message 2', 'this is message 1']


 exit
